<a href="https://colab.research.google.com/github/DARKINOO/Pytorch_practice/blob/main/pytorch_autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates PyTorch's autograd feature, which automatically calculates gradients.

In [18]:
import torch
# Imports the torch library

First, we create a tensor `x` with `requires_grad=True`. This tells PyTorch to track computations involving this tensor so that gradients can be calculated later.

In [19]:
x = torch.tensor(3.0, requires_grad=True) # if we want derivative of any tensor, we write requiregrad=True
# Creates a tensor with a value of 3.0 and enables gradient tracking

Next, we define a new tensor `y` as a function of `x` (`y = x^2`). PyTorch builds a computation graph to track this operation.

In [20]:
y = x**2
# Defines y as x squared. PyTorch records this operation in the computation graph.
print(x)
print(y)
# Prints the value of x and y

tensor(3., requires_grad=True)
tensor(9., grad_fn=<PowBackward0>)


Now, we call `y.backward()`. This is the core of autograd. It computes the gradients of `y` with respect to all tensors that have `requires_grad=True` in the computation graph (in this case, just `x`). The gradient is accumulated in the `.grad` attribute of the respective tensors.

In [21]:
y.backward() #calc gradients automatically
# Computes the gradient of y with respect to x and stores it in x.grad

Finally, we access `x.grad` to see the computed gradient. Since y = x^2, the derivative dy/dx is 2x. With x = 3, the gradient is 2 * 3 = 6.

In [22]:
x.grad
# Displays the computed gradient of x

tensor(6.)

###Computation of perceptron using autograd

In [25]:
x = torch.tensor(6.7) #Input feature
y = torch.tensor(0.0) #True label (binary)

w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [26]:
w

tensor(1., requires_grad=True)

In [27]:
b

tensor(0., requires_grad=True)

In [30]:
#Binary Cross-Entropy Loss for scaler
def binary_cross_entropy_loss(prediction,target):
  epsilon = 1e-8 #to prevent log(0)
  prediction = torch.clamp(prediction, epsilon, 1-epsilon)
  loss = -(target*torch.log(prediction) + (1-target)*torch.log(1-prediction))
  return loss

In [31]:
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [32]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [33]:
loss = binary_cross_entropy_loss(y_pred,y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [34]:
loss.backward()

In [35]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


###Autograd on Vectors

In [36]:
x = torch.tensor([1.0,2.0,3.0], requires_grad=True)
x

tensor([1., 2., 3.], requires_grad=True)

In [37]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [38]:
y.backward()

In [39]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

####clearning gradients

In [41]:

# when we run again y.backward(), it stores gradients , they dont erase automatically
# so we have to clear prev gradients in multiple passes
x.grad.zero_()

tensor([0., 0., 0.])

In [ ]:
# disable gradient tracking

### Disabling Gradient Tracking

Disable gradient tracking for efficiency during inference or when not needed. Three methods:

#### 1. Setting `requires_grad=False`

Set `requires_grad` to `False` when creating a tensor or for an existing one.

In [42]:
# Method 1: requires_grad=False

x = torch.tensor([1., 2., 3.], requires_grad=True)
x_no_grad = torch.tensor([4., 5., 6.], requires_grad=False)
print(f'{x.requires_grad=}')
print(f'{x_no_grad.requires_grad=}')

x.requires_grad_(False)
print(f'{x.requires_grad=}')

x.requires_grad=True
x_no_grad.requires_grad=False
x.requires_grad=False


#### 2. Using `.detach()`

Create a new tensor detached from the graph. Original tensor still tracks gradients.

In [43]:
# Method 2: .detach()

x = torch.tensor([1., 2., 3.], requires_grad=True)
x_detached = x.detach()
print(f'{x.requires_grad=}')
print(f'{x_detached.requires_grad=}')

x.requires_grad=True
x_detached.requires_grad=False


#### 3. Using `torch.no_grad()`

Context manager to disable gradient tracking for a block of code.

In [44]:
# Method 3: torch.no_grad()

x = torch.tensor([1., 2., 3.], requires_grad=True)
with torch.no_grad():
    y = (x**2).mean()
    print(f'{y.requires_grad=}')

y_outside = (x**3).mean()
print(f'{y_outside.requires_grad=}')

y.requires_grad=False
y_outside.requires_grad=True
